In [1]:
import numpy as np
import pandas as pd
import glob
import os
from pathlib import Path
from itertools import product
import regex as re

In [2]:

def compute_lps(pattern):
    # Longest Proper Prefix that is suffix array
    lps = [0] * len(pattern)

    prefi = 0
    for i in range(1, len(pattern)):
        
        while prefi and pattern[i] != pattern[prefi]:
            prefi = lps[prefi - 1]

        if pattern[prefi] == pattern[i]:
            prefi += 1
            lps[i] = prefi


    return lps



def kmp(pattern, text):
    match_indices = []
    count = 0
    pattern_lps = compute_lps(pattern)

    patterni = 0
    for i, ch in enumerate(text):
        
        while patterni and pattern[patterni] != ch:
            patterni = pattern_lps[patterni - 1]

        if pattern[patterni] == ch:
            if patterni == len(pattern) - 1:
                match_indices.append(i - patterni)
                count += 1
                patterni = pattern_lps[patterni]
            
            else:
                patterni += 1

    return match_indices, count


In [3]:
def remove_impossible_patterns(pattern_list):

    patterns_cleaned = list(pattern_list)

    for sub in list(patterns_cleaned):
        if 'RR' in sub:
            patterns_cleaned.remove(sub)
            continue
        elif 'LL' in sub:
            patterns_cleaned.remove(sub)
            continue
        elif 'JJ' in sub:
            patterns_cleaned.remove(sub)
            continue
        elif 'rr' in sub:
            patterns_cleaned.remove(sub)
            continue
        elif 'll' in sub:
            patterns_cleaned.remove(sub)
            continue
        elif 'jj' in sub:
            patterns_cleaned.remove(sub)
            continue

    return patterns_cleaned

In [4]:

p3 = [''.join(item) for item in product('RLJ', 'RrLlJj', 'RrLlJj')]
p3_clean = remove_impossible_patterns(p3)


list_patterns_set = [[], [l for l in 'RrLlJj'], [''.join(item) for item in product('RLJ', 'RrLlJj')], p3_clean]
list_best_freq= [[], [], [], []]


In [5]:

df = pd.read_csv('/Users/caldera/Documents/EPFL/GAP/Project/Projet-Gameplay/wip/kevin/data/keylogs/ground_truth/2022-11-23-12_13_44.csv')
df_trim = df[(df['KEY'] == 'Key.left') | (df['KEY'] == 'Key.right') | (df['KEY'] == '\'d\'')]

df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'R'
df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'UP'), 'action'] = 'r'

df_trim.loc[(df_trim['KEY'] == 'Key.left') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'L'
df_trim.loc[(df_trim['KEY'] == 'Key.left') & (df_trim['STATUS'] == 'UP'), 'action'] = 'l'

df_trim.loc[(df_trim['KEY'] == '\'d\'') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'J'
df_trim.loc[(df_trim['KEY'] == '\'d\'') & (df_trim['STATUS'] == 'UP'), 'action'] = 'j'
df_trim

/var/folders/0y/g4f502v57jd_s8zx0025dl3w0000gn/T/ipykernel_87199/601260875.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'R'


,FRAME,KEY,STATUS,action
4,346,Key.right,DOWN,R
5,491,'d',DOWN,J
6,517,'d',UP,j
7,519,Key.right,UP,r
8,534,Key.left,DOWN,L
...,...,...,...,...
2928,67243,'d',DOWN,J
2929,67301,'d',UP,j
2930,67356,'d',DOWN,J
2931,67363,'d',UP,j


In [6]:
candidates = dict()
best_timeframe_size = 0

for second_tick in np.linspace(3, 10, 15): #3., 3.5, 4, ..., 9.5, 10.
    n_frame = second_tick*60
    slice_size = int(df_trim.FRAME.max()/n_frame)


    freq = pd.DataFrame(index=list_patterns_set[3], columns=['count', 'frequency'])
    freq['count'] = 0
    freq['frequency'] = 0

    for period in range(slice_size):
        slice = df_trim[(df_trim.FRAME >= period*n_frame) & (df_trim.FRAME < (period+1)*n_frame)]

        str = slice.action.str.cat()

        for item in freq.index:
            freq.loc[item, 'count'] += kmp(item, str)[1]

    n_pattern = freq['count'].sum()

    freq = freq.loc[freq['count'] > 0]    

    for item in freq.index:
        if n_pattern > 0:
            freq.loc[item, 'frequency'] = freq.loc[item, 'count']/n_pattern * 100
        else:
            freq.loc[item, 'frequency'] = 0

    candidates[second_tick] = freq.sort_values(by=['count'], ascending=False).head(15)

    print(second_tick, n_pattern)

list_patterns_set.append([''.join(item) for item in product(freq.index, 'RrLlJj')])
list_best_freq.append(candidates)
print(len(freq.index))
candidates

3.0 877
3.5 895
4.0 924
4.5 963
5.0 966
5.5 977
6.0 1002
6.5 996
7.0 1010
7.5 1025
8.0 1036
8.5 1045
9.0 1051
9.5 1044
10.0 1055
43


{3.0:      count  frequency
 JjJ    145  16.533637
 Jjr    118  13.454960
 RJj    102  11.630559
 LlR     64   7.297605
 RrR     47   5.359179
 JjR     41   4.675029
 RrJ     36   4.104903
 LlL     30   3.420753
 Jrj     29   3.306727
 Jjl     28   3.192702
 RrL     28   3.192702
 LJj     24   2.736602
 RJr     24   2.736602
 JRj     23   2.622577
 JLj     17   1.938426,
 3.5:      count  frequency
 JjJ    151  16.871508
 Jjr    114  12.737430
 RJj    101  11.284916
 LlR     64   7.150838
 RrR     54   6.033520
 JjR     43   4.804469
 RrJ     33   3.687151
 LlL     32   3.575419
 RrL     29   3.240223
 Jjl     29   3.240223
 Jrj     27   3.016760
 RJr     25   2.793296
 LJj     24   2.681564
 JRj     23   2.569832
 LlJ     20   2.234637,
 4.0:      count  frequency
 JjJ    159  17.207792
 RJj    123  13.311688
 Jjr    117  12.662338
 LlR     63   6.818182
 RrR     51   5.519481
 JjR     46   4.978355
 RrJ     34   3.679654
 LlL     32   3.463203
 RrL     29   3.138528
 Jjl     28   3.0

In [7]:
candidates = dict()
best_timeframe_size = 0

for second_tick in np.linspace(3, 10, 15): #3., 3.5, 4, ..., 9.5, 10.
    n_frame = second_tick*60
    slice_size = int(df_trim.FRAME.max()/n_frame)


    freq = pd.DataFrame(index=list_patterns_set[4], columns=['count', 'frequency'])
    freq['count'] = 0
    freq['frequency'] = 0

    for period in range(slice_size):
        slice = df_trim[(df_trim.FRAME >= period*n_frame) & (df_trim.FRAME < (period+1)*n_frame)]

        str = slice.action.str.cat()

        for item in freq.index:
            freq.loc[item, 'count'] += kmp(item, str)[1]

    n_pattern = freq['count'].sum()

    freq = freq.loc[freq['count'] > 0]    

    for item in freq.index:
        if n_pattern > 0:
            freq.loc[item, 'frequency'] = freq.loc[item, 'count']/n_pattern * 100
        else:
            freq.loc[item, 'frequency'] = 0

    candidates[second_tick] = freq.sort_values(by=['count'], ascending=False).head(15)

    print(second_tick, n_pattern)

list_patterns_set.append([''.join(item) for item in product(freq.index, 'RrLlJj')])
list_best_freq.append(candidates)
print(len(freq.index))
candidates

3.0 738
3.5 774
4.0 825
4.5 868
5.0 879
5.5 908
6.0 932
6.5 921
7.0 944
7.5 961
8.0 973
8.5 996
9.0 996
9.5 992
10.0 1006
90


{3.0:       count  frequency
 JjJj    110  14.905149
 RJjr     53   7.181572
 JjrL     42   5.691057
 RJjJ     36   4.878049
 JjrR     30   4.065041
 LlLl     27   3.658537
 RrRr     26   3.523035
 LlRJ     25   3.387534
 LlRr     25   3.387534
 RrJj     22   2.981030
 LJjl     20   2.710027
 RJrj     20   2.710027
 JjlR     18   2.439024
 JjRJ     18   2.439024
 JjRr     17   2.303523,
 3.5:       count  frequency
 JjJj    116  14.987080
 RJjr     45   5.813953
 JjrL     43   5.555556
 JjrR     40   5.167959
 RJjJ     36   4.651163
 LlRJ     31   4.005168
 LlLl     27   3.488372
 RrRr     26   3.359173
 LlRr     23   2.971576
 RrJj     22   2.842377
 LJjl     21   2.713178
 RJrj     21   2.713178
 JjRr     21   2.713178
 JjlR     19   2.454780
 RrRJ     19   2.454780,
 4.0:       count  frequency
 JjJj    129  15.636364
 RJjr     63   7.636364
 JjrL     46   5.575758
 RJjJ     45   5.454545
 JjrR     37   4.484848
 LlRJ     32   3.878788
 LlLl     29   3.515152
 JjRr     24   2.909091

In [8]:
candidates = dict()
best_timeframe_size = 0

for second_tick in np.linspace(3, 10, 15): #3., 3.5, 4, ..., 9.5, 10.
    n_frame = second_tick*60
    slice_size = int(df_trim.FRAME.max()/n_frame)


    freq = pd.DataFrame(index=list_patterns_set[5], columns=['count', 'frequency'])
    freq['count'] = 0
    freq['frequency'] = 0

    for period in range(slice_size):
        slice = df_trim[(df_trim.FRAME >= period*n_frame) & (df_trim.FRAME < (period+1)*n_frame)]

        str = slice.action.str.cat()

        for item in freq.index:
            freq.loc[item, 'count'] += kmp(item, str)[1]

    n_pattern = freq['count'].sum()

    freq = freq.loc[freq['count'] > 0]    

    for item in freq.index:
        if n_pattern > 0:
            freq.loc[item, 'frequency'] = freq.loc[item, 'count']/n_pattern * 100
        else:
            freq.loc[item, 'frequency'] = 0

    candidates[second_tick] = freq.sort_values(by=['count'], ascending=False).head(15)

    print(second_tick, n_pattern)

list_patterns_set.append([''.join(item) for item in product(freq.index, 'RrLlJj')])
list_best_freq.append(candidates)
print(len(freq.index))
candidates

3.0 588
3.5 645
4.0 699
4.5 756
5.0 783
5.5 811
6.0 831
6.5 844
7.0 867
7.5 885
8.0 898
8.5 921
9.0 925
9.5 940
10.0 952
165


{3.0:        count  frequency
 JjJjJ     41   6.972789
 JjJjr     29   4.931973
 JjrLl     27   4.591837
 RJjJj     21   3.571429
 JjrRJ     19   3.231293
 RJjrL     17   2.891156
 LlRJj     16   2.721088
 RJjrR     13   2.210884
 LlLlL     13   2.210884
 LJjlR     12   2.040816
 RrRrR     12   2.040816
 RrJjR     11   1.870748
 JjRrJ     10   1.700680
 LlRrR     10   1.700680
 RrLlR      9   1.530612,
 3.5:        count  frequency
 JjJjJ     54   8.372093
 JjrLl     35   5.426357
 JjJjr     29   4.496124
 JjrRJ     26   4.031008
 LlRJj     21   3.255814
 RJjJj     21   3.255814
 RJjrR     17   2.635659
 LlLlL     15   2.325581
 RrRrR     14   2.170543
 LJjlR     13   2.015504
 RrLlR     13   2.015504
 RJjrL     13   2.015504
 RrJjR     12   1.860465
 RrRJj     11   1.705426
 RJrjR     11   1.705426,
 4.0:        count  frequency
 JjJjJ     61   8.726753
 RJjJj     35   5.007153
 JjJjr     32   4.577969
 JjrLl     32   4.577969
 LlRJj     25   3.576538
 JjrRJ     23   3.290415
 RJjrR  

In [9]:
candidates = dict()
best_timeframe_size = 0

for second_tick in np.linspace(3, 10, 15): #3., 3.5, 4, ..., 9.5, 10.
    n_frame = second_tick*60
    slice_size = int(df_trim.FRAME.max()/n_frame)


    freq = pd.DataFrame(index=list_patterns_set[6], columns=['count', 'frequency'])
    freq['count'] = 0
    freq['frequency'] = 0

    for period in range(slice_size):
        slice = df_trim[(df_trim.FRAME >= period*n_frame) & (df_trim.FRAME < (period+1)*n_frame)]

        str = slice.action.str.cat()

        for item in freq.index:
            freq.loc[item, 'count'] += kmp(item, str)[1]

    n_pattern = freq['count'].sum()

    freq = freq.loc[freq['count'] > 0]    

    for item in freq.index:
        if n_pattern > 0:
            freq.loc[item, 'frequency'] = freq.loc[item, 'count']/n_pattern * 100
        else:
            freq.loc[item, 'frequency'] = 0

    candidates[second_tick] = freq.sort_values(by=['count'], ascending=False).head(15)

    print(second_tick, n_pattern)

list_patterns_set.append([''.join(item) for item in product(freq.index, 'RrLlJj')])
list_best_freq.append(candidates)
print(len(freq.index))
candidates

3.0 500
3.5 551
4.0 615
4.5 675
5.0 705
5.5 734
6.0 772
6.5 773
7.0 804
7.5 829
8.0 845
8.5 875
9.0 882
9.5 888
10.0 907
244


{3.0:         count  frequency
 JjJjJj     31        6.2
 RJjrLl     14        2.8
 JjrRJj     14        2.8
 LlLlLl     13        2.6
 JjJjrL     12        2.4
 JjrLlR     12        2.4
 LlRJjr     11        2.2
 RJjJjr      9        1.8
 JjRrJj      9        1.8
 RJjrRJ      8        1.6
 JjRJjJ      8        1.6
 RJjJjJ      8        1.6
 LJjlRr      7        1.4
 JjrLlL      7        1.4
 RrJjRr      7        1.4,
 3.5:         count  frequency
 JjJjJj     38   6.896552
 JjrRJj     20   3.629764
 JjrLlR     18   3.266788
 LlLlLl     14   2.540835
 RJjrRJ     14   2.540835
 JjJjrL     13   2.359347
 RJjrLl     12   2.177858
 JjRrJj     10   1.814882
 RJjJjJ      9   1.633394
 LlRJjr      9   1.633394
 JjRJjJ      8   1.451906
 RJjJjr      8   1.451906
 JjrLlL      7   1.270417
 RrJjRr      7   1.270417
 RrLlRr      6   1.088929,
 4.0:         count  frequency
 JjJjJj     47   7.642276
 RJjJjJ     18   2.926829
 JjrRJj     17   2.764228
 RJjrLl     16   2.601626
 JjJjrL     15   2.43

In [10]:
candidates = dict()
best_timeframe_size = 0

for second_tick in np.linspace(3, 10, 15): #3., 3.5, 4, ..., 9.5, 10.
    n_frame = second_tick*60
    slice_size = int(df_trim.FRAME.max()/n_frame)


    freq = pd.DataFrame(index=list_patterns_set[7], columns=['count', 'frequency'])
    freq['count'] = 0
    freq['frequency'] = 0

    for period in range(slice_size):
        slice = df_trim[(df_trim.FRAME >= period*n_frame) & (df_trim.FRAME < (period+1)*n_frame)]

        str = slice.action.str.cat()

        for item in freq.index:
            freq.loc[item, 'count'] += kmp(item, str)[1]

    n_pattern = freq['count'].sum()

    freq = freq.loc[freq['count'] > 0]    

    for item in freq.index:
        if n_pattern > 0:
            freq.loc[item, 'frequency'] = freq.loc[item, 'count']/n_pattern * 100
        else:
            freq.loc[item, 'frequency'] = 0

    candidates[second_tick] = freq.sort_values(by=['count'], ascending=False).head(15)

    print(second_tick, n_pattern)

list_patterns_set.append([''.join(item) for item in product(freq.index, 'RrLlJj')])
list_best_freq.append(candidates)
print(len(freq.index))
candidates

3.0 394
3.5 449
4.0 516
4.5 585
5.0 626
5.5 651
6.0 692
6.5 706
7.0 736
7.5 763
8.0 786
8.5 805
9.0 820
9.5 828
10.0 866
348


{3.0:          count  frequency
 JjJjJjJ      9   2.284264
 JjJjJjr      7   1.776650
 LlLlLlL      7   1.776650
 JjrRJjr      7   1.776650
 RJjrRJj      7   1.776650
 JjrLlRJ      7   1.776650
 RJjJjJj      6   1.522843
 JjRrJjR      5   1.269036
 JjRJjJj      5   1.269036
 JjJjrLl      5   1.269036
 LlLlLlR      5   1.269036
 LlRrRrR      5   1.269036
 JjrLlLl      4   1.015228
 RrLJjlR      4   1.015228
 JjrRJjJ      4   1.015228,
 3.5:          count  frequency
 JjJjJjJ     15   3.340757
 JjrLlRJ     12   2.672606
 JjJjrLl     10   2.227171
 RJjrRJj      9   2.004454
 LlLlLlL      8   1.781737
 JjrRJjJ      8   1.781737
 JjJjJjr      8   1.781737
 JjrRJjr      7   1.559020
 JjRJjJj      6   1.336303
 LlLlLlR      5   1.113586
 JjRrJjR      5   1.113586
 RJjJjJj      5   1.113586
 JjrLlLl      4   0.890869
 RrLlRrR      4   0.890869
 RJjrLlJ      4   0.890869,
 4.0:          count  frequency
 JjJjJjJ     18   3.488372
 JjrRJjr     15   2.906977
 RJjJjJj     12   2.325581
 JjJjJjr   

In [11]:
candidates = dict()
best_timeframe_size = 0

for second_tick in np.linspace(3, 10, 15): #3., 3.5, 4, ..., 9.5, 10.
    n_frame = second_tick*60
    slice_size = int(df_trim.FRAME.max()/n_frame)


    freq = pd.DataFrame(index=list_patterns_set[8], columns=['count', 'frequency'])
    freq['count'] = 0
    freq['frequency'] = 0

    for period in range(slice_size):
        slice = df_trim[(df_trim.FRAME >= period*n_frame) & (df_trim.FRAME < (period+1)*n_frame)]

        str = slice.action.str.cat()

        for item in freq.index:
            freq.loc[item, 'count'] += kmp(item, str)[1]

    n_pattern = freq['count'].sum()

    freq = freq.loc[freq['count'] > 0]    

    for item in freq.index:
        if n_pattern > 0:
            freq.loc[item, 'frequency'] = freq.loc[item, 'count']/n_pattern * 100
        else:
            freq.loc[item, 'frequency'] = 0

    candidates[second_tick] = freq.sort_values(by=['count'], ascending=False).head(15)

    print(second_tick, n_pattern)

list_patterns_set.append([''.join(item) for item in product(freq.index, 'RrLlJj')])
list_best_freq.append(candidates)
print(len(freq.index))
candidates

3.0 305
3.5 355
4.0 427
4.5 503
5.0 555
5.5 580
6.0 620
6.5 642
7.0 659
7.5 698
8.0 724
8.5 752
9.0 758
9.5 777
10.0 815
432


{3.0:           count  frequency
 LlLlLlLl      7   2.295082
 JjJjJjJj      6   1.967213
 JjrLlRJj      6   1.967213
 LlRrRrRr      4   1.311475
 RrJjRrJj      4   1.311475
 JjJjrLlR      3   0.983607
 RjJjJjJj      3   0.983607
 JjRJjJjJ      3   0.983607
 LlRrLJjl      3   0.983607
 JjrRJjJj      3   0.983607
 RJjrLlLl      3   0.983607
 JjrLlLlL      3   0.983607
 JjJjrLJj      3   0.983607
 JjrJRjrR      3   0.983607
 RJjrLlJj      3   0.983607,
 3.5:           count  frequency
 JjJjJjJj      9   2.535211
 JjrLlRJj      8   2.253521
 LlLlLlLl      7   1.971831
 JjJjrLlR      6   1.690141
 JjrRJjJj      5   1.408451
 JjrLlLlL      4   1.126761
 JjrRJjrR      4   1.126761
 JjJjJjrL      4   1.126761
 JRjrRJjJ      3   0.845070
 LlRrLJjl      3   0.845070
 RJjrRJjr      3   0.845070
 RJjrRJjJ      3   0.845070
 RrJjJjJj      3   0.845070
 RJjrLlJj      3   0.845070
 RrJjRrJj      3   0.845070,
 4.0:           count  frequency
 JjJjJjJj     14   3.278689
 JjrLlRJj      7   1.639344
 Ll

In [12]:
p3 = [''.join(item) for item in product('RLJ', 'RrLlJj', 'RrLlJj')]
p3_clean = remove_impossible_patterns(p3)


list_patterns_set = [[], [l for l in 'RrLlJj'], [''.join(item) for item in product('RLJ', 'RrLlJj')], p3_clean]

In [14]:
p3 = [''.join(item) for item in product('RLJ', 'RrLlJj', 'RrLlJj')]
p3_clean = remove_impossible_patterns(p3)
columns = pd.MultiIndex.from_product([np.linspace(2, 10, 17), ['count', 'frequency', 'best_freq']], names=["window (s)", "stat"])


list_patterns_set = [[], [l for l in 'RrLlJj'], remove_impossible_patterns([''.join(item) for item in product('RLJ', 'RrLlJj')]), p3_clean]

freqs = pd.DataFrame(columns=columns)
best_freq_locs = dict()

for i in range(3,11): # on fait pour des patters de longueur 3 à 10
    print('starting size', i)
    freq = pd.DataFrame(0, index=list_patterns_set[i], columns=columns)
    freq.loc[:, pd.IndexSlice[:, 'best_freq']] = False    
    best_freq_loc = dict(zip(freq.index, [[0,0]]*len(freq.index))) # va servir a stocker la fenetre pour laquelle on a la plus haute frequence pour un pattern donné ('frequency','window')

    for window in np.linspace(2, 10, 17): #2., 2.5, 3, ..., 9.5, 10. --> On teste des intervalles à la demi-seconde
        n_frame = window*60 #nb de frame a considerer = secondes*FPS
        n_slice = int(df_trim.FRAME.max()/n_frame) #nombre de période de taille "window"

        for period in range(n_slice): #pour chaque période (cad pour chaque bloc contenant n_frame)
            slice = df_trim[(df_trim.FRAME >= period*n_frame) & (df_trim.FRAME < (period+1)*n_frame)] #on SLICE

            str = slice.action.str.cat() #on recup la chaine de caractere des actions sur laquelle on va faire la detection

            for pattern in freq.index:
                freq.loc[pattern, (window, 'count')] += kmp(pattern, str)[1] #pour chaque pattern on compte les occurences

        n_pattern = freq[window, 'count'].sum() #nb total de pattern detecté pour une window

        #freq = freq.loc[freq[window, 'count'] > 0]   #on garde que les patterns apparaissants 

        # on calcule et record si necessaire les frequences et la plus haute frequence
        if n_pattern > 0:
            for pattern in freq.index:
                freq.loc[pattern, (window, 'frequency')] = freq.loc[pattern, (window, 'count')]/n_pattern * 100
                
                if freq.loc[pattern, (window, 'frequency')] > best_freq_loc[pattern][0]:
                    freq.loc[pattern, (best_freq_loc[pattern][1], 'best_freq')] = False
                    freq.loc[pattern, (window, 'best_freq')] = True

                    best_freq_loc[pattern][0] = freq.loc[pattern, (window, 'frequency')]
                    best_freq_loc[pattern][1] = window


        #candidates[window] = freq.sort_values(by=['count'], ascending=False).head(15)

        print(window, n_pattern)

    print(len(freq))
    #on concat
    freqs = pd.concat([freqs, freq])
    best_freq_locs = best_freq_locs | best_freq_loc

    #on crée le set de patterns de taille k+1 : si un pattern de taille k n'apparait pas dans la string,
    #alors aucun pattern de taille k+1 commencant par ses k caracteres n'existe non plus
    list_patterns_set.append([''.join(pattern) for pattern in product(freq.index, 'RrLlJj')]) 

freqs


starting size 3
2.0 724
2.5 812
3.0 877
3.5 895
4.0 924
4.5 963
5.0 966
5.5 977
6.0 1002
6.5 996
7.0 1010
7.5 1025
8.0 1036
8.5 1045
9.0 1051
9.5 1044
10.0 1055
75
starting size 4
2.0 570
2.5 663
3.0 738
3.5 775
4.0 826
4.5 869
5.0 879
5.5 909
6.0 932
6.5 922
7.0 945
7.5 961
8.0 974
8.5 997
9.0 997
9.5 993
10.0 1006
450
starting size 5


/var/folders/0y/g4f502v57jd_s8zx0025dl3w0000gn/T/ipykernel_87199/4004850351.py:52: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  freqs = pd.concat([freqs, freq])


2.0 404


KeyboardInterrupt: 

In [ ]:
best_freq_locs

{'RrR': [21.51082450483648, 8.0],
 'RrL': [21.51082450483648, 8.0],
 'Rrl': [21.51082450483648, 8.0],
 'RrJ': [21.51082450483648, 8.0],
 'Rrj': [21.51082450483648, 8.0],
 'RLR': [21.51082450483648, 8.0],
 'RLr': [21.51082450483648, 8.0],
 'RLl': [21.51082450483648, 8.0],
 'RLJ': [21.51082450483648, 8.0],
 'RLj': [21.51082450483648, 8.0],
 'RlR': [21.51082450483648, 8.0],
 'Rlr': [21.51082450483648, 8.0],
 'RlL': [21.51082450483648, 8.0],
 'RlJ': [21.51082450483648, 8.0],
 'Rlj': [21.51082450483648, 8.0],
 'RJR': [21.51082450483648, 8.0],
 'RJr': [21.51082450483648, 8.0],
 'RJL': [21.51082450483648, 8.0],
 'RJl': [21.51082450483648, 8.0],
 'RJj': [21.51082450483648, 8.0],
 'RjR': [21.51082450483648, 8.0],
 'Rjr': [21.51082450483648, 8.0],
 'RjL': [21.51082450483648, 8.0],
 'Rjl': [21.51082450483648, 8.0],
 'RjJ': [21.51082450483648, 8.0],
 'LRr': [21.51082450483648, 8.0],
 'LRL': [21.51082450483648, 8.0],
 'LRl': [21.51082450483648, 8.0],
 'LRJ': [21.51082450483648, 8.0],
 'LRj': [21.51

In [ ]:
test = pd.DataFrame(np.random.rand(75,51), index=list_patterns_set[3], columns=columns)
test.loc[:, pd.IndexSlice[:, 'best_freq']] = False

test


window (s)      2.0                           2.5                       \
stat           count frequency best_freq     count frequency best_freq   
RrR         0.532716  0.724652     False  0.205734  0.677177     False   
RrL         0.155429  0.905780     False  0.006163  0.258924     False   
Rrl         0.697857  0.555208     False  0.131229  0.257042     False   
RrJ         0.854042  0.833901     False  0.645913  0.769345     False   
Rrj         0.627589  0.194587     False  0.641063  0.733559     False   
..               ...       ...       ...       ...       ...       ...   
JjR         0.657214  0.352800     False  0.629716  0.814478     False   
Jjr         0.548137  0.504835     False  0.592647  0.027060     False   
JjL         0.348482  0.181991     False  0.323099  0.344631     False   
Jjl         0.059628  0.141724     False  0.487819  0.410962     False   
JjJ         0.680696  0.593659     False  0.313395  0.581089     False   

window (s)      3.0                           3.5   ...      8.5       9.0   \
stat           count frequency best_freq     count  ... best_freq     count   
RrR         0.613655  0.605398     False  0.626813  ...     False  0.377178   
RrL         0.759151  0.600518     False  0.687077  ...     False  0.673197   
Rrl         0.124477  0.050964     False  0.981393  ...     False  0.478230   
RrJ         0.876132  0.889787     False  0.221544  ...     False  0.784279   
Rrj         0.911786  0.670043     False  0.066815  ...     False  0.888654   
..               ...       ...       ...       ...  ...       ...       ...   
JjR         0.485389  0.876572     False  0.151186  ...     False  0.589928   
Jjr         0.328628  0.236849     False  0.326873  ...     False  0.070891   
JjL         0.652555  0.965233     False  0.909467  ...     False  0.638987   
Jjl         0.679950  0.656749     False  0.699211  ...     False  0.448622   
JjJ         0.427443  0.320604     False  0.097884  ...     False  0.978683   

window (s)                          9.5                           10.0  \
stat       frequency best_freq     count frequency best_freq     count   
RrR         0.021112     False  0.739825  0.658102     False  0.887132   
RrL         0.943954     False  0.399200  0.330410     False  0.891984   
Rrl         0.580585     False  0.363587  0.175724     False  0.375267   
RrJ         0.237961     False  0.230051  0.370313     False  0.693218   
Rrj         0.247556     False  0.488450  0.819935     False  0.745396   
..               ...       ...       ...       ...       ...       ...   
JjR         0.371174     False  0.324303  0.039246     False  0.280769   
Jjr         0.569842     False  0.153573  0.345910     False  0.061280   
JjL         0.427796     False  0.550444  0.604553     False  0.170072   
Jjl         0.726349     False  0.007967  0.078428     False  0.588141   
JjJ         0.957586     False  0.285123  0.470939     False  0.884996   

window (s)                      
stat       frequency best_freq  
RrR         0.164517     False  
RrL         0.676098     False  
Rrl         0.004170     False  
RrJ         0.752207     False  
Rrj         0.810476     False  
..               ...       ...  
JjR         0.968275     False  
Jjr         0.175119     False  
JjL         0.358054     False  
Jjl         0.123474     False  
JjJ         0.379891     False  

[75 rows x 51 columns]

In [ ]:
test.loc[test[:, 'count'] > 0.5]   #on garde que les patterns apparaissants 


window (s)      2.0                           2.5                       \
stat           count frequency best_freq     count frequency best_freq   
RrR         0.532716  0.724652     False  0.205734  0.677177     False   
Rrl         0.697857  0.555208     False  0.131229  0.257042     False   
RrJ         0.854042  0.833901     False  0.645913  0.769345     False   
Rrj         0.627589  0.194587     False  0.641063  0.733559     False   
RLR         0.984064  0.531006     False  0.470420  0.890412     False   
RLl         0.709289  0.305428     False  0.934860  0.849305     False   
RlR         0.931391  0.782661     False  0.685914  0.054074     False   
RlL         0.635471  0.315524     False  0.131157  0.983504     False   
Rlj         0.691390  0.710851     False  0.235231  0.786932     False   
RJR         0.883014  0.676763     False  0.335445  0.511001     False   
RJr         0.950452  0.927625     False  0.894566  0.991998     False   
Rjr         0.690308  0.945528     False  0.752784  0.728967     False   
Rjl         0.777490  0.843378     False  0.231094  0.005329     False   
RjJ         0.998873  0.456549     False  0.231232  0.980767     False   
LRr         0.885475  0.427785     False  0.701935  0.648988     False   
LRL         0.798613  0.838588     False  0.975274  0.153449     False   
LRl         0.840460  0.286449     False  0.417870  0.085173     False   
LRJ         0.700073  0.344011     False  0.538555  0.794358     False   
LRj         0.884582  0.545423     False  0.623698  0.763548     False   
LrR         0.598532  0.786823     False  0.815687  0.668641     False   
LrL         0.635377  0.459982     False  0.016687  0.046313     False   
Llr         0.647640  0.546244     False  0.856354  0.608924     False   
LlJ         0.996691  0.433077     False  0.944668  0.685735     False   
LJR         0.863526  0.208583     False  0.584724  0.427845     False   
LjL         0.921997  0.912912     False  0.447390  0.697887     False   
Ljl         0.765398  0.106701     False  0.070405  0.465862     False   
JRr         0.974384  0.395009     False  0.530285  0.575963     False   
JRJ         0.565702  0.688727     False  0.789058  0.645304     False   
JRj         0.662243  0.836335     False  0.045877  0.422304     False   
JrR         0.529906  0.366974     False  0.416185  0.219677     False   
JrJ         0.759172  0.020882     False  0.941086  0.962611     False   
Jrj         0.812879  0.393700     False  0.104254  0.124387     False   
JLl         0.829632  0.493947     False  0.306970  0.373940     False   
JLj         0.623299  0.571247     False  0.010083  0.344042     False   
Jlr         0.518258  0.587256     False  0.519509  0.241792     False   
JlJ         0.615920  0.749795     False  0.326790  0.276514     False   
JjR         0.657214  0.352800     False  0.629716  0.814478     False   
Jjr         0.548137  0.504835     False  0.592647  0.027060     False   
JjJ         0.680696  0.593659     False  0.313395  0.581089     False   

window (s)      3.0                           3.5   ...      8.5       9.0   \
stat           count frequency best_freq     count  ... best_freq     count   
RrR         0.613655  0.605398     False  0.626813  ...     False  0.377178   
Rrl         0.124477  0.050964     False  0.981393  ...     False  0.478230   
RrJ         0.876132  0.889787     False  0.221544  ...     False  0.784279   
Rrj         0.911786  0.670043     False  0.066815  ...     False  0.888654   
RLR         0.034402  0.310499     False  0.005845  ...     False  0.093379   
RLl         0.878277  0.370951     False  0.886755  ...     False  0.600241   
RlR         0.263946  0.501979     False  0.609728  ...     False  0.078925   
RlL         0.305307  0.788935     False  0.715438  ...     False  0.893155   
Rlj         0.953796  0.150420     False  0.803687  ...     False  0.263578   
RJR         0.395779  0.400974     False  0.374903  ...     False  0.297080   
RJr         0.942

In [ ]:

df = pd.read_csv('/Users/caldera/Documents/EPFL/GAP/Project/Projet-Gameplay/wip/kevin/data/keylogs/ground_truth/2022-11-25-12_13_37.csv')
df_trim = df[(df['KEY'] == 'Key.left') | (df['KEY'] == 'Key.right') | (df['KEY'] == 'Key.space')]

#R -> Down, r -> Up
df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'R'
df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'UP'), 'action'] = 'r'

df_trim.loc[(df_trim['KEY'] == 'Key.left') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'L'
df_trim.loc[(df_trim['KEY'] == 'Key.left') & (df_trim['STATUS'] == 'UP'), 'action'] = 'l'

df_trim.loc[(df_trim['KEY'] == 'Key.space') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'J'
df_trim.loc[(df_trim['KEY'] == 'Key.space') & (df_trim['STATUS'] == 'UP'), 'action'] = 'j'
df_trim.action.str.cat()

/var/folders/0y/g4f502v57jd_s8zx0025dl3w0000gn/T/ipykernel_51138/249272645.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'R'


'RJjrRrJRjrJRrRjrJLljLJljLlRJjJjJjJjJjJLjrJljRJjrLlRJjJjrLJjRlrRrRJjJjJjrLJljRJjrLlRJrjRJjrLlJjRJLrljLJljRrRJjrLlLJjlRLJrjlRrRJjJjJrjJRjJjrRJjrRJjJjJjJjJjrRJjrJjRrRrRrRrRrRrRJjJjJjJjJjJjrRJjJjJjJrjRrRrLlRrRrRrJjJjJjRrRrRrJjJjLlRJjrJRjrJjJjRJjrLlRJjrRJrjRrRJrjLlRJrjRJrjRJjrRrRrRJjrLJljRrJRrjRrLJjlJjRJjJjJjJjLrJjJjJjJjlRrRJjrRJjrRrRJjJjJjJjJjJjJrjLJjJjlRJjrLJljLlRJjrLJRljrRrRJjrJjRJjJjrLlRrLlRJjrLlRrLlRrRrJRjJjrLlJjRrJjLlJjLJljRrJRrjLlRrRrRrJjRrJjLlRrJjRrRrJLljRJjrRrLlRJjJjJjJjJjJjJjJjJjrLJjlRrLlJjJjJjJjJjJjJjLlJRrjRJjrJjJjJjJjRJjrLJjlJjJjJjJjRrRJjrLlLlLlRJjJjJjrLlRJjrRrJjRrRrRrJjLlJjRJrjRrRJjJjrRrRrRrLlJjRrJLjlLJjlRrLJjlRrJRjrLJjlJjLJljLlLlLlLlLlLlLlLlLlLlJjRrJLjlJLljLJjlLJjlJjRrLJljJjLlRrLJljJLRljrLlLJljLlJLjlJLjlLJjlRJjrLJjlRrJRjrLlLJljLJjlJjRrLJjlJLljRrJjRJjJjJjJjJjJjJjrLJjJjJjlRJjrRrLlLlLlRJjJjJjrJRjrRJjrRrRrLlLlRJjLrlRrLJjlRJjrRJjrRrRJjrRJrjRrLJjlRrRJjrLlRrJRjrRrRJjJjrLlRJjrJRjrRrRrRrLlRJjrLJjlRrRrLJjlRJjJjrLlRrLlRJjJjJjrRJjrLJjlRrJLjlRJjrRrJjJLjlRrRJjrRJjrLJjlRrJRjrLlJjRJjrRJrjJjL